In [ ]:
##Upload files, stored locally

In [5]:
from google.colab import files
files.upload()

Saving CS98XClassificationTest.csv to CS98XClassificationTest (2).csv
Saving CS98XClassificationTrain.csv to CS98XClassificationTrain (1).csv


{'CS98XClassificationTest (2).csv': b'Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop\r\n454,Pump It,The Black Eyed Peas,2005,154,93,65,-3,75,74,213,1,18,72\r\n455,"Circle of Life - From ""The Lion King""/Soundtrack Version",Elton John,1994,161,39,30,-15,11,14,292,26,3,59\r\n456,We Are The Champions - Remastered 2011,Queen,1977,64,46,27,-7,12,18,179,38,3,76\r\n457,Insomnia - Radio Edit,Faithless,2010,127,92,71,-9,37,53,216,6,4,50\r\n458,This Eve of Parting,John Hartford,2018,115,46,56,-12,21,34,153,18,3,44\r\n459,You Keep Me Hangin On,Kim Wilde,1986,123,68,69,-12,14,71,255,8,3,60\r\n460,Mandy,Barry Manilow,1974,104,46,31,-9,14,32,213,42,3,69\r\n461,Collette,Billy Fury,1982,155,44,55,-10,10,73,112,82,4,27\r\n462,Against All Odds (Take a Look at Me Now) - 2016 Remaster,Phil Collins,2016,116,50,58,-7,13,11,206,20,3,74\r\n463,Highway to Hell,AC/DC,1979,116,91,57,-5,16,42,208,6,13,83\r\n464,In The Still Of The Night,The Five Satins,2010,110,32,44,-10,11,20,184,88,4,44\r\n4

In [97]:
##Install necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

In [98]:
##Read training and test sets

training = pd.read_csv("CS98XClassificationTrain.csv")
test_comp = pd.read_csv("CS98XClassificationTest.csv")

In [99]:
##Genre counts
training['top genre'].value_counts()

,count
top genre,
adult standards,68
album rock,66
dance pop,61
brill building pop,16
glam rock,16
...,...
bow pop,1
australian rock,1
boogaloo,1


In [9]:
##86 genres, many with very few instances
##Find any null

In [100]:
training['top genre'].isnull().sum()

15

In [101]:
##Remove rare genres, threshold value was changed and this notebook rerun which yielded the differing results, as presented in report
# 7 found to be the ideal threshold
genre_counts = training['top genre'].value_counts()
rare_genres = genre_counts[genre_counts<4].index
training = training[~training['top genre'].isin(rare_genres)]

In [102]:
##Label encoding
le = LabelEncoder()
training['top genre'] = le.fit_transform(training['top genre'])

In [103]:
##Seperate target from features

y = training['top genre']
X = training[['bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch', 'pop']]

In [104]:
##Train-test split

trainX, valX, trainY, valY = train_test_split(X, y, test_size = 0.2, random_state=42)

In [105]:
##Standardize the data
scaler = StandardScaler()
trainX_scaled = scaler.fit_transform(trainX)
valX_scaled = scaler.transform(valX)


In [106]:
##Build Logistic Regression
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(trainX_scaled, trainY)
validationPred_log = log_reg.predict(valX_scaled)
val_accuracy_log = accuracy_score(valY, validationPred_log)
print("Logistic Regression accuracy score: ", val_accuracy_log)

Logistic Regression accuracy score:  0.4383561643835616


In [107]:
##Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(trainX, trainY)
validationPred_RF = rf_model.predict(valX)
val_accuracy_RF = accuracy_score(valY, validationPred_RF)
print("Random Forest accuracy score:",val_accuracy_RF)

Random Forest accuracy score: 0.410958904109589


In [108]:
##Support Vector Machine
svm_model = SVC(kernel='rbf', C=1.0, random_state=42)
svm_model.fit(trainX_scaled, trainY)
validation_pred_SVC = svm_model.predict(valX_scaled)
val_accuracy_SVC = accuracy_score(valY, validation_pred_SVC)
print("Support Vector Machine Accuracy", val_accuracy_SVC)

Support Vector Machine Accuracy 0.410958904109589


In [109]:
##Decision Tree
dt_model = DecisionTreeClassifier(max_depth=None, random_state=42)
dt_model.fit(trainX, trainY)
validation_pred_DT = dt_model.predict(valX)
val_accuracy_DT = accuracy_score(valY, validation_pred_DT)
print("Decision Tree Accuracy:", val_accuracy_DT)

Decision Tree Accuracy: 0.1643835616438356


In [110]:
##Ensemble
voting_model = VotingClassifier(estimators=[('lr', log_reg), ('rf', rf_model), ('svc', svm_model)],voting='hard')
voting_model.fit(trainX_scaled, trainY)
validation_pred_Vote = voting_model.predict(valX_scaled)
val_accuracy_Vote = accuracy_score(valY, validation_pred_Vote)
print(val_accuracy_Vote)

0.4383561643835616


In [112]:
##Set up test set
testId = test_comp['Id']
testX = test_comp.drop(columns=['Id', 'title', 'artist', 'year'])
testX = scaler.fit_transform(testX)
Xtest_scaled = scaler.fit_transform(testX)

In [113]:
##Retrain Models

Ytrain = training['top genre']
Xtrain = training[['bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch', 'pop']]

Xtrain_scaled = scaler.fit_transform(Xtrain)

log_reg.fit(trainX_scaled, trainY)
rf_model.fit(trainX, trainY)
svm_model.fit(trainX_scaled, trainY)
dt_model.fit(trainX, trainY)
voting_model.fit(trainX_scaled, trainY)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=1000,
                                                 random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [114]:
##Make predictions
logPred = log_reg.predict(Xtest_scaled)
rfPred = rf_model.predict(testX)
svmPred = svm_model.predict(Xtest_scaled)
dtPred = dt_model.predict(testX)
votingPred = voting_model.predict(Xtest_scaled)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [115]:
##Decode predictions
logPred = le.inverse_transform(logPred)
rfPred = le.inverse_transform(rfPred)
svmPred = le.inverse_transform(svmPred)
dtPred = le.inverse_transform(dtPred)
votingPred = le.inverse_transform(votingPred)

In [116]:
##Predictions to datafranes
logPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': logPred
})

rfPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': rfPred
})

svmPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': svmPred
})

dtPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': dtPred
})

votingPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': votingPred
})

In [117]:
##Predictions to datafranes
logPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': logPred
})

rfPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': rfPred
})

svmPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': svmPred
})

dtPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': dtPred
})

votingPredDF=pd.DataFrame({
    'Id': testId,
    'top genre': votingPred
})

In [118]:
##Output to csv

logPredDF.to_csv('/content/logPredDF.csv', index=False)
rfPredDF.to_csv('/content/rfPredDFCHANGED.csv', index=False)
svmPredDF.to_csv('/content/svmPredDF.csv', index=False)
dtPredDF.to_csv('/content/dtPredDF.csv', index=False)
votingPredDF.to_csv('/content/votingPredDF.csv', index=False)

In [119]:
##Download results for submission to kaggle
files.download('/content/logPredDF.csv')
files.download('/content/rfPredDFCHANGED.csv')
files.download('/content/svmPredDF.csv')
files.download('/content/dtPredDF.csv')
files.download('/content/votingPredDF.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [127]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
texlive is already the newest version (2021.20220204-1).
texlive-latex-extra is already the newest version (2021.20220204-1).
texlive-xetex is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [132]:
!jupyter nbconvert "/content/drive/My Drive/Colab Notebooks/ClassificationFinal.ipynb" --to pdf

[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/ClassificationFinal.ipynb to pdf
[NbConvertApp] Writing 127214 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 101945 bytes to /content/drive/My Drive/Colab Notebooks/ClassificationFinal.pdf


In [131]:
!jupyter nbconvert --to notebook "/content/drive/My Drive/Colab Notebooks/ClassificationFinal.ipynb"

[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/ClassificationFinal.ipynb to notebook
[NbConvertApp] Writing 121729 bytes to /content/drive/My Drive/Colab Notebooks/ClassificationFinal.nbconvert.ipynb
